In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

# collect()
- collect 함수는 액션 함수 : RDD 또는 DataFrame의 모든 요소를 로컬 머신으로 수집합니다.
- 주요 특징: 전체 데이터를 수집하여 리스트로 반환, 메모리 사용 주의.
- 사용법: 작은 데이터셋에서 디버깅 및 확인 용도로 사용.
- 대규모 데이터셋 대안: take(), show(), limit() 등의 함수 사용.

In [2]:
dept = [("Finance",10), \
    ("Marketing",20), \
    ("Sales",30), \
    ("IT",40) \
  ]
deptColumns = ["dept_name","dept_id"]
deptDF = spark.createDataFrame(data=dept, schema = deptColumns)
deptDF.show(truncate=False)

+---------+-------+
|dept_name|dept_id|
+---------+-------+
|Finance  |10     |
|Marketing|20     |
|Sales    |30     |
|IT       |40     |
+---------+-------+



In [3]:
deptDF.collect()

[Row(dept_name='Finance', dept_id=10),
 Row(dept_name='Marketing', dept_id=20),
 Row(dept_name='Sales', dept_id=30),
 Row(dept_name='IT', dept_id=40)]

# withColumn

In [4]:
data = [('James','','Smith','1991-04-01','M',3000),
  ('Michael','Rose','','2000-05-19','M',4000),
  ('Robert','','Williams','1978-09-05','M',4000),
  ('Maria','Anne','Jones','1967-12-01','F',4000),
  ('Jen','Mary','Brown','1980-02-17','F',-1)
]
columns = ["firstname","middlename","lastname","dob","gender","salary"]

df = spark.createDataFrame(data=data, schema=columns)
df.show(truncate=False)

+---------+----------+--------+----------+------+------+
|firstname|middlename|lastname|dob       |gender|salary|
+---------+----------+--------+----------+------+------+
|James    |          |Smith   |1991-04-01|M     |3000  |
|Michael  |Rose      |        |2000-05-19|M     |4000  |
|Robert   |          |Williams|1978-09-05|M     |4000  |
|Maria    |Anne      |Jones   |1967-12-01|F     |4000  |
|Jen      |Mary      |Brown   |1980-02-17|F     |-1    |
+---------+----------+--------+----------+------+------+



In [5]:
df.printSchema()

root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: long (nullable = true)



In [6]:
from pyspark.sql.functions import col

In [7]:
df.withColumn("salary", col("salary").cast("Integer")).printSchema()

root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)



In [8]:
# 기존 변수 타입은 그대로 두면서 새로운 타입으로 파생 변수 생성 가능
df.withColumn("salary_t", col("salary").cast("Integer")).printSchema()

root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: long (nullable = true)
 |-- salary_t: integer (nullable = true)



In [9]:
# 연산까지만 적으면 수행이 안 됨. show() 라는 액션 명령어가 있어야 수행됨. 
df.withColumn("salary_t", col("salary").cast("Integer")*1400).show()

+---------+----------+--------+----------+------+------+--------+
|firstname|middlename|lastname|       dob|gender|salary|salary_t|
+---------+----------+--------+----------+------+------+--------+
|    James|          |   Smith|1991-04-01|     M|  3000| 4200000|
|  Michael|      Rose|        |2000-05-19|     M|  4000| 5600000|
|   Robert|          |Williams|1978-09-05|     M|  4000| 5600000|
|    Maria|      Anne|   Jones|1967-12-01|     F|  4000| 5600000|
|      Jen|      Mary|   Brown|1980-02-17|     F|    -1|   -1400|
+---------+----------+--------+----------+------+------+--------+



In [10]:
# 일관된 값으로 새로운 컬럼 만들기 
from pyspark.sql.functions import lit
df.withColumn('country', lit('korea')).show()

+---------+----------+--------+----------+------+------+-------+
|firstname|middlename|lastname|       dob|gender|salary|country|
+---------+----------+--------+----------+------+------+-------+
|    James|          |   Smith|1991-04-01|     M|  3000|  korea|
|  Michael|      Rose|        |2000-05-19|     M|  4000|  korea|
|   Robert|          |Williams|1978-09-05|     M|  4000|  korea|
|    Maria|      Anne|   Jones|1967-12-01|     F|  4000|  korea|
|      Jen|      Mary|   Brown|1980-02-17|     F|    -1|  korea|
+---------+----------+--------+----------+------+------+-------+



In [11]:
# RDD는 불변의 성격을 갖고 있기 때문에, 컬럼을 추가하더라도 원본 df가 변하지 않는다. 
df.show()

+---------+----------+--------+----------+------+------+
|firstname|middlename|lastname|       dob|gender|salary|
+---------+----------+--------+----------+------+------+
|    James|          |   Smith|1991-04-01|     M|  3000|
|  Michael|      Rose|        |2000-05-19|     M|  4000|
|   Robert|          |Williams|1978-09-05|     M|  4000|
|    Maria|      Anne|   Jones|1967-12-01|     F|  4000|
|      Jen|      Mary|   Brown|1980-02-17|     F|    -1|
+---------+----------+--------+----------+------+------+



In [12]:
# df2 라는 새로운 RDD에 넣어주기 
df2 = df.withColumn('country', lit('korea'))

In [13]:
df2.withColumnRenamed('salary', '월급').show()

+---------+----------+--------+----------+------+----+-------+
|firstname|middlename|lastname|       dob|gender|월급|country|
+---------+----------+--------+----------+------+----+-------+
|    James|          |   Smith|1991-04-01|     M|3000|  korea|
|  Michael|      Rose|        |2000-05-19|     M|4000|  korea|
|   Robert|          |Williams|1978-09-05|     M|4000|  korea|
|    Maria|      Anne|   Jones|1967-12-01|     F|4000|  korea|
|      Jen|      Mary|   Brown|1980-02-17|     F|  -1|  korea|
+---------+----------+--------+----------+------+----+-------+



In [14]:
df2 = df2.drop('country')

In [15]:
df2.show()

+---------+----------+--------+----------+------+------+
|firstname|middlename|lastname|       dob|gender|salary|
+---------+----------+--------+----------+------+------+
|    James|          |   Smith|1991-04-01|     M|  3000|
|  Michael|      Rose|        |2000-05-19|     M|  4000|
|   Robert|          |Williams|1978-09-05|     M|  4000|
|    Maria|      Anne|   Jones|1967-12-01|     F|  4000|
|      Jen|      Mary|   Brown|1980-02-17|     F|    -1|
+---------+----------+--------+----------+------+------+



# 파이썬 판다스와 연결

In [16]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [17]:
import pandas as pd

In [19]:
import pandas as pd
pandas_df = pd.DataFrame({'a': [1,2,3], 'b' : [2,3,4]})
spark_df = spark.createDataFrame(pandas_df)

In [21]:
pandas_df

,a,b
0,1,2
1,2,3
2,3,4


In [23]:
spark_df.show()

+---+---+
|  a|  b|
+---+---+
|  1|  2|
|  2|  3|
|  3|  4|
+---+---+



# 스타벅스 데이터 크롤링

In [20]:
import requests
url = "https://www.starbucks.co.kr/store/getStore.do?r=3PS46TOGMA"
payload = {"in_biz_cds": "0",
"in_scodes": "0",
"ins_lat": "37.56682",
"ins_lng": "126.97865",
"search_text": "",
"p_sido_cd": "01",
"p_gugun_cd": "",
"isError": "true",
"in_distance": "0",
"in_biz_cd": "",
"iend": "1000",
"searchType": "C",
"set_date": "",
"rndCod": "NTM40TCD0S",
"all_store": "0",
"T03": "0",
"T01": "0",
"T27": "0",
"T12": "0",
"T09": "0",
"T30": "0",
"T05": "0",
"T22": "0",
"T21": "0",
"T10": "0",
"T36": "0",
"T43": "0",
"T48": "0",
"Z9999": "0",
"P02": "0",
"P10": "0",
"P50": "0",
"P20": "0",
"P60": "0",
"P30": "0",
"P70": "0",
"P40": "0",
"P80": "0",
"whcroad_yn": "0",
"P90": "0",
"P01": "0",
"new_bool": "0",}

In [24]:
r = requests.post(url, data=payload)

In [26]:
# starbucks_df = spark.createDataFrame(pd.DataFrame(star))
# 위 코드 실행했을 때는 결측치 때문에 DF 생성하지 못하는 오류 발생 
star = pd.DataFrame(r.json()['list'])

In [29]:
star

,seq,sido_cd,sido_nm,gugun_cd,gugun_nm,code_order,view_yn,store_num,sido,gugun,...,p01,t05,t30,t36,t27,t29,t43,t48,z9999,p02
0,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
1,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
2,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
3,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
4,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
608,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
609,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
610,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0


In [30]:
?star.dropna

Signature:
star.dropna(
    *,
    axis: 'Axis' = 0,
    how: 'AnyAll | lib.NoDefault' = <no_default>,
    thresh: 'int | lib.NoDefault' = <no_default>,
    subset: 'IndexLabel | None' = None,
    inplace: 'bool' = False,
    ignore_index: 'bool' = False,
) -> 'DataFrame | None'
Docstring:
Remove missing values.

See the :ref:`User Guide <missing_data>` for more on which values are
considered missing, and how to work with missing data.

Parameters
----------
axis : {0 or 'index', 1 or 'columns'}, default 0
    Determine if rows or columns which contain missing values are
    removed.

    * 0, or 'index' : Drop rows which contain missing values.
    * 1, or 'columns' : Drop columns which contain missing value.

    Only a single axis is allowed.

how : {'any', 'all'}, default 'any'
    Determine if row or column is removed from DataFrame, when we have
    at least one NA or all NA.

    * 'any' : If any NA values are present, drop that row or column.
    * 'all' : If all values are NA,

In [31]:
# 결측치가 600개 이상인 컬럼만 삭제하겠다는 옵션
star.dropna(axis=1, thresh=600) 

,seq,p_pro_seq,p_sido_cd,p_gugun_cd,s_code,s_name,tel,fax,sido_code,sido_name,...,p01,t05,t30,t36,t27,t29,t43,t48,z9999,p02
0,0,0,,,1509,역삼아레나빌딩,1522-3232,02-568-3763,01,서울,...,0,0,0,0,0,0,0,0,0,0
1,0,0,,,1434,논현역사거리,1522-3232,02-3442-3673,01,서울,...,0,0,0,0,0,0,0,0,0,0
2,0,0,,,1595,신사역성일빌딩,1522-3232,02-547-3859,01,서울,...,0,0,0,0,0,0,0,0,0,0
3,0,0,,,1527,국기원사거리,1522-3232,02-568-3669,01,서울,...,0,0,0,0,0,0,0,0,0,0
4,0,0,,,1468,대치재경빌딩,1522-3232,02-568-3705,01,서울,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607,0,0,,,838,사가정역,1522-3232,02-435-8823,01,서울,...,0,0,0,0,0,0,0,0,0,0
608,0,0,,,493,상봉역,1522-3232,02-433-8486,01,서울,...,0,0,0,0,0,0,0,0,0,0
609,0,0,,,1668,묵동,1522-3232,02-971-3937,01,서울,...,0,0,0,0,0,0,0,0,0,0
610,0,0,,,2002,양원역,1522-3232,02-433-4308,01,서울,...,0,0,0,0,0,0,0,0,0,0


In [34]:
# 결측치 제거한 데이터로 df 만들기 
starbucks_df = spark.createDataFrame(pd.DataFrame(star.dropna(axis=1)))

In [36]:
starbucks_df.limit(1).show()

+---+---------+---------+----------+------+--------------+---------+-----------+---------+---------+----------+----------+-------------------------------+--------------------+--------+-----------+-------+-------+-----------+------------+--------------+---------------+----------------+--------+--------+--------+----------+--------+--------+---------+--------------------+----------+---------+----+------+----+--------+---------+------+-------+--------+---------+-------------+-------------------------------+---------+-----------------------+----------+-----+-------+---------+-----------+---------------+---------+----------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+-----+---+
|seq|p_pro_seq|p_sido_cd|p_gugun_cd|s_code|        s_name|      tel|        fax|sido_code|sido_name|gugun_code|gugun_name|                           addr|         theme_state|new_bool|search_text|ins_lat|ins_lng|in_distance|out_distance|all_search_cnt|ad

In [41]:
# 특정 컬럼만 뽑기 
starbucks_df.select("s_code", "s_name", "open_dt", "espresso", "addr").show(3,truncate=False) 

+------+--------------+--------+--------+------------------------------------------+
|s_code|s_name        |open_dt |espresso|addr                                      |
+------+--------------+--------+--------+------------------------------------------+
|1509  |역삼아레나빌딩|20190613|        |서울특별시 강남구 역삼동 721-13 아레나빌딩|
|1434  |논현역사거리  |20181123|        |서울특별시 강남구 논현동 142-2 정일빌딩   |
|1595  |신사역성일빌딩|20191219|        |서울특별시 강남구 논현동 18-4 성일빌딩    |
+------+--------------+--------+--------+------------------------------------------+
only showing top 3 rows



In [57]:
# 여러 컬럼 이름 바꾸기 
starbucks_df.withColumnsRenamed({'s_code': '매장코드', 's_name': '매장이름', 'open_dt':'개장일', 'addr':'주소'}).\
            select("매장코드", "매장이름", "개장일", "주소").\
            show(5, truncate=False)

+--------+--------------+--------+------------------------------------------+
|매장코드|매장이름      |개장일  |주소                                      |
+--------+--------------+--------+------------------------------------------+
|1509    |역삼아레나빌딩|20190613|서울특별시 강남구 역삼동 721-13 아레나빌딩|
|1434    |논현역사거리  |20181123|서울특별시 강남구 논현동 142-2 정일빌딩   |
|1595    |신사역성일빌딩|20191219|서울특별시 강남구 논현동 18-4 성일빌딩    |
|1527    |국기원사거리  |20190731|서울특별시 강남구 역삼동 648-22 동찬빌딩  |
|1468    |대치재경빌딩  |20190214|서울특별시 강남구 대치동 599 대원빌딩     |
+--------+--------------+--------+------------------------------------------+
only showing top 5 rows



In [60]:
# 다른 방식으로 컬럼명 바꾸기 
# 새롭게 지정한 컬럼명으로 새로운 데이터 프레임 만드는 함수
# ?starbucks_df.toDF

star_df = starbucks_df.select('s_name', "s_code", "open_dt", "espresso", "addr")
star_df2= star_df.toDF(*['매장이름', '매장코드', '개장일','espresso', '주소'])
star_df2.show(1)

+--------------+--------+--------+--------+-------------------------------+
|      매장이름|매장코드|  개장일|espresso|                           주소|
+--------------+--------+--------+--------+-------------------------------+
|역삼아레나빌딩|    1509|20190613|        |서울특별시 강남구 역삼동 721...|
+--------------+--------+--------+--------+-------------------------------+
only showing top 1 row



In [61]:
from pyspark.sql.functions import split

In [62]:
star_df2 = star_df2.withColumn('split_col', split(star_df2['주소'], " "))

In [64]:
star_df2.show(3, truncate=False)

+--------------+--------+--------+--------+------------------------------------------+------------------------------------------------+
|매장이름      |매장코드|개장일  |espresso|주소                                      |split_col                                       |
+--------------+--------+--------+--------+------------------------------------------+------------------------------------------------+
|역삼아레나빌딩|1509    |20190613|        |서울특별시 강남구 역삼동 721-13 아레나빌딩|[서울특별시, 강남구, 역삼동, 721-13, 아레나빌딩]|
|논현역사거리  |1434    |20181123|        |서울특별시 강남구 논현동 142-2 정일빌딩   |[서울특별시, 강남구, 논현동, 142-2, 정일빌딩]   |
|신사역성일빌딩|1595    |20191219|        |서울특별시 강남구 논현동 18-4 성일빌딩    |[서울특별시, 강남구, 논현동, 18-4, 성일빌딩]    |
+--------------+--------+--------+--------+------------------------------------------+------------------------------------------------+
only showing top 3 rows



In [66]:
# 주소 split한 결과는 array 값으로 저장돼 있음을 알 수 있다. 
star_df2.printSchema()

root
 |-- 매장이름: string (nullable = true)
 |-- 매장코드: string (nullable = true)
 |-- 개장일: string (nullable = true)
 |-- espresso: string (nullable = true)
 |-- 주소: string (nullable = true)
 |-- split_col: array (nullable = true)
 |    |-- element: string (containsNull = false)



In [72]:
# array 내 인덱스 설정으로 특정 컬럼만 뽑기 
star_df2.select(star_df2["split_col"][1]).show(3)

+------------+
|split_col[1]|
+------------+
|      강남구|
|      강남구|
|      강남구|
+------------+
only showing top 3 rows



In [75]:
# 새 컬럼으로 저장하기 
star_df2 = star_df2.withColumn('구', star_df2["split_col"][1])
star_df2.show(3)

+--------------+--------+--------+--------+-------------------------------+-------------------------------+------+
|      매장이름|매장코드|  개장일|espresso|                           주소|                      split_col|    구|
+--------------+--------+--------+--------+-------------------------------+-------------------------------+------+
|역삼아레나빌딩|    1509|20190613|        |서울특별시 강남구 역삼동 721...|[서울특별시, 강남구, 역삼동,...|강남구|
|  논현역사거리|    1434|20181123|        |서울특별시 강남구 논현동 142...|[서울특별시, 강남구, 논현동,...|강남구|
|신사역성일빌딩|    1595|20191219|        |서울특별시 강남구 논현동 18-...|[서울특별시, 강남구, 논현동,...|강남구|
+--------------+--------+--------+--------+-------------------------------+-------------------------------+------+
only showing top 3 rows



In [78]:
star_df2.groupby('구')\
        .count()\
        .sort('count', ascending=False)\
        .show()

+--------+-----+
|      구|count|
+--------+-----+
|  강남구|   91|
|    중구|   52|
|  서초구|   48|
|영등포구|   43|
|  종로구|   39|
|  송파구|   37|
|  마포구|   36|
|  강서구|   27|
|  용산구|   24|
|서대문구|   21|
|  광진구|   19|
|  양천구|   17|
|  강동구|   17|
|  성북구|   15|
|  노원구|   14|
|  은평구|   14|
|  구로구|   14|
|  성동구|   14|
|  금천구|   13|
|동대문구|   13|
+--------+-----+
only showing top 20 rows



In [79]:
star_df2 = star_df2.drop('espresso', 'split_col')

# createOrReplaceTempView
- `.createOrReplaceTempView` 함수는 PySpark DataFrame을 SQL 쿼리로 조회할 수 있는 임시 테이블로 등록하는 데 사용됩니다.
- 이 임시 테이블은 세션 내에서만 존재하며, 세션이 종료되면 사라집니다.
- 이 함수는 기존에 같은 이름의 임시 테이블이 존재하는 경우 그 테이블을 대체합니다.

In [80]:
star_df2.createOrReplaceTempView("starbucks")

In [81]:
spark.sql("select * from starbucks").show()

+-----------------+--------+--------+-------------------------------+------+
|         매장이름|매장코드|  개장일|                           주소|    구|
+-----------------+--------+--------+-------------------------------+------+
|   역삼아레나빌딩|    1509|20190613|서울특별시 강남구 역삼동 721...|강남구|
|     논현역사거리|    1434|20181123|서울특별시 강남구 논현동 142...|강남구|
|   신사역성일빌딩|    1595|20191219|서울특별시 강남구 논현동 18-...|강남구|
|     국기원사거리|    1527|20190731|서울특별시 강남구 역삼동 648...|강남구|
|     대치재경빌딩|    1468|20190214|서울특별시 강남구 대치동 599...|강남구|
|         봉은사역|    1640|20200528|서울특별시 강남구 삼성동 108...|강남구|
|   압구정윤성빌딩|    1650|20200529|서울특별시 강남구 신사동 592...|강남구|
|     코엑스별마당|    1626|20200416|서울특별시 강남구 삼성동 159...|강남구|
|  삼성역섬유센터R|    1438|20181123|서울특별시 강남구 대치동 944...|강남구|
|          압구정R|    1411|20180831| 서울특별시 강남구 신사동 621-1|강남구|
|          수서역R|    1389|20180627|서울특별시 강남구 수서동 715...|강남구|
|    양재강남빌딩R|    1404|20180824|서울특별시 강남구 도곡동 956...|강남구|
|    선릉동신빌딩R|    1355|20180330|서울특별시 강남구 삼성동 141...|강남구|
|   봉은사로선정릉|    1348|20180306|서울특별시 강남

In [91]:
spark.sql("""
        SELECT `구`, COUNT(*) AS CNT
        FROM starbucks
        GROUP BY `구`
        ORDER BY CNT DESC
        """).show()

+--------+---+
|      구|CNT|
+--------+---+
|  강남구| 91|
|    중구| 52|
|  서초구| 48|
|영등포구| 43|
|  종로구| 39|
|  송파구| 37|
|  마포구| 36|
|  강서구| 27|
|  용산구| 24|
|서대문구| 21|
|  광진구| 19|
|  양천구| 17|
|  강동구| 17|
|  성북구| 15|
|  노원구| 14|
|  은평구| 14|
|  구로구| 14|
|  성동구| 14|
|  금천구| 13|
|동대문구| 13|
+--------+---+
only showing top 20 rows



In [ ]:
star_df2.groupby('구')\
        .count()\
        .sort('count', ascending=False)\
        .show()

In [15]:
bicycle = spark.\
    read.\
    csv("hdfs://192.168.0.160:8020/encore/tpss_bcycl_od_statnhm_202001.csv", header=True, inferSchema=True, encoding='euc-kr')

ERROR:root:KeyboardInterrupt while sending command.                 (0 + 2) / 2]
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 